In [11]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("ltp") # all RAM subjects
exp = 'ltpFR2' 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
np.unique(df.experiment)
exp_df = df[df.experiment==exp]
np.shape(exp_df)
exp_df
print(str(len(np.unique(exp_df.subject)))+' '+exp+' subjects')

array(['NiclsCourierClosedLoop', 'NiclsCourierReadOnly', 'VFFR',
       'ltpDBOY1', 'ltpFR2', 'ltpRepFR', 'prelim'], dtype=object)

(2470, 9)

,all_events,experiment,import_type,math_events,original_session,session,subject,subject_alias,task_events
0,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,0,0,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
1,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,1,1,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
2,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,10,10,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
3,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,11,11,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
4,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,12,12,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
...,...,...,...,...,...,...,...,...,...
2594,protocols/ltp/subjects/LTP393/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP393/experiments/ltpF...,5,5,LTP393,LTP393,protocols/ltp/subjects/LTP393/experiments/ltpF...
2595,protocols/ltp/subjects/LTP393/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP393/experiments/ltpF...,6,6,LTP393,LTP393,protocols/ltp/subjects/LTP393/experiments/ltpF...
2596,protocols/ltp/subjects/LTP393/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP393/experiments/ltpF...,7,7,LTP393,LTP393,protocols/ltp/subjects/LTP393/experiments/ltpF...
2597,protocols/ltp/subjects/LTP393/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP393/experiments/ltpF...,8,8,LTP393,LTP393,protocols/ltp/subjects/LTP393/experiments/ltpF...


127 ltpFR2 subjects


In [ ]:
def EEGReadDFRow(row):
    from cmlreaders import CMLReader, get_data_index
    rd = row._asdict() # this takes df and takes values from 1 row as a dict
    return CMLReader(rd['subject'], rd['experiment'], rd['session'], \
                     montage=0, localization=0) # these two always 0s for EEG

for row in exp_df.itertuples(): #sub_df.itertuples():   
    sub = row.subject; session = row.session; exp = row.experiment

    reader = EEGReadDFRow(row)
    evs = reader.load('task_events')
    evs_math = reader.load('all_events')
    evs_math = evs_math[(evs_math.type=='PROB')]   # get math problems only 
    import ipdb; ipdb.set_trace()

--Call--
> /home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/IPython/core/displayhook.py(252)__call__()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.



ipdb>  u


> <ipython-input-23-3a1eeec9f3f2>(14)<module>()
     12     evs_math = reader.load('all_events')
     13     evs_math = evs_math[(evs_math.type=='PROB')]
---> 14     import ipdb; ipdb.set_trace()



ipdb>  l


      9 
     10     reader = EEGReadDFRow(row)
     11     evs = reader.load('task_events')
     12     evs_math = reader.load('all_events')
     13     evs_math = evs_math[(evs_math.type=='PROB')]
---> 14     import ipdb; ipdb.set_trace()



...

2       [7, 8, 9]
3       [3, 8, 9]
4       [2, 6, 3]
32      [6, 6, 9]
33      [3, 4, 1]
          ...    
1323    [6, 5, 7]
1324    [6, 9, 8]
1325    [4, 8, 1]
1326    [4, 1, 1]
1327    [7, 7, 4]
Name: test, Length: 393, dtype: object


ipdb>  evs_math.test.unique()


*** TypeError: unhashable type: 'list'


ipdb>  np.unique(evs_math.test)


array([list([1, 1, 4]), list([1, 1, 5]), list([1, 1, 7]), list([1, 1, 9]),
       list([1, 2, 1]), list([1, 2, 5]), list([1, 2, 8]), list([1, 3, 2]),
       list([1, 3, 3]), list([1, 3, 5]), list([1, 3, 7]), list([1, 3, 8]),
       list([1, 4, 1]), list([1, 4, 6]), list([1, 4, 9]), list([1, 5, 1]),
       list([1, 5, 2]), list([1, 5, 3]), list([1, 5, 4]), list([1, 5, 6]),
       list([1, 5, 9]), list([1, 6, 1]), list([1, 6, 2]), list([1, 6, 5]),
       list([1, 6, 7]), list([1, 7, 4]), list([1, 7, 7]), list([1, 7, 9]),
       list([1, 8, 1]), list([1, 8, 3]), list([1, 8, 4]), list([1, 8, 7]),
       list([1, 8, 8]), list([1, 8, 9]), list([1, 9, 1]), list([1, 9, 2]),
       list([1, 9, 4]), list([1, 9, 6]), list([2, 1, 1]), list([2, 1, 4]),
       list([2, 1, 5]), list([2, 1, 6]), list([2, 2, 1]), list([2, 2, 2]),
       list([2, 2, 4]), list([2, 2, 5]), list([2, 2, 7]), list([2, 2, 8]),
       list([2, 2, 9]), list([2, 3, 1]), list([2, 3, 2]), list([2, 3, 7]),
       list([2, 3, 8]), l